In [1]:
import re
import pandas as pd

file_path = '../data/2_ds2_trimmed.csv'

df = pd.read_csv(file_path)
df = df.dropna(subset=['tag', 'lyrics'])

In [ ]:
# Used this to test different chunks of data

import pandas as pd
column_names = pd.read_csv(file_path, nrows=0).columns

chunk_size = 10**5  # Define your chunk size
chunk_number = 3     # Specify which chunk you want (e.g., 2nd chunk here)

# Calculate the number of rows to skip
rows_to_skip = chunk_size * (chunk_number - 1)+1

# Load the next chunk with skiprows
df = pd.read_csv(file_path, skiprows=rows_to_skip, nrows=chunk_size,  header=None,names=column_names)
df['lyrics']=df['lyrics'].astype(str)
# Display a sample to verify
df.head()

In [ ]:
df.head()

In [2]:
def clean_lyrics(text):
    # Remove all text within square brackets
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'\{.*?\}', '', text)
    text = re.sub(r'\(.*?\)', '', text)
    # Remove any remaining newline characters
    text = text.replace('\n', ' ')
    # Remove leading/trailing whitespace
    text = text.strip()
    return text

In [3]:
# clean out nonuseful words
df['lyrics']=df['lyrics'].apply(clean_lyrics)
# Keep only lyrics with 20 or more words
clean_df = df[df['lyrics'].str.split().str.len() >= 20]

# note gets rid of short songs, zero lyrics, intrumentals, and non lric words such as intro chorus etc

clean_df.head()

,title,tag,artist,year,views,features,lyrics,id
0,Revelation,rock,Zardonic,2018,6680,{},"Try to do it like this, you won't get it Try t...",3849758
1,Robitussin,rb,OPENPAD,2017,94,"{""Rossi Rock""}","Saucalini: Baby what you want, what you need?...",3387226
2,Tornado,rock,The Hillbilly Moon Explosion,2007,23,{},Oh she started to rock like a tornado A-whirli...,5407232
3,All On Me,rap,J Blaze,2022,2,"{""Aj Slimm""}",The things that I've been through I'm fighting...,7761795
5,Kevin Gates,rap,Kevin Gates,2018,490,{},"We had some bumps along the way, but the rewar...",3785780


In [4]:
# Sample of these short lyrics to inspect
#sampled_short_lyrics = short_lyrics.sample(15, random_state=15)

# Display the results
#for index, lyric in enumerate(sampled_short_lyrics['lyrics'], 1):
#    print(f"Lyrics Sample {index}:\n{lyric}\n{'-'*50}")


In [ ]:
from langdetect import detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException

# Ensure consistent results from langdetect
DetectorFactory.seed = 0

def is_english(text):
    try:
        # Detect language of text
        return detect(text) == 'en'
    except LangDetectException:
        # If language detection fails, treat it as non-English
        return False


In [ ]:
# #This was a check to see if it sorted correctly
# # Note this can take a long time with large inputs

# # Sample 100 random lyrics along with titles
# sampled_lyrics = df[['title', 'lyrics']].sample(100, random_state=5)

# # Check each sampled lyric for English language
# sampled_lyrics['is_english'] = sampled_lyrics['lyrics'].apply(is_english)

# # Filter for non-English lyrics in the sampled set
# non_english_lyrics = sampled_lyrics[sampled_lyrics['is_english'] == False]

# # Display non-English sampled lyrics
# print("Sampled Lyrics Detected as Non-English:")
# for index, row in non_english_lyrics.iterrows():
#     print(f"Title: {row['title']}\nLyrics:\n{row['lyrics']}\n{'-'*50}")


In [ ]:
# note this can take a very large time recommend trying your computer with a sample first.

clean_df = clean_df[clean_df['lyrics'].apply(is_english)]


In [ ]:
clean_df.to_csv("../data/3_ds3_cleaned.csv", index=False)
